In the first question, I have suggested that we don't really know the number of clusters as increasing K caused more clusters. In this question, I seek answer to how many clusters are there given the data and its properties. 

In [1]:
from __future__ import (absolute_import, print_function, division)
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd
import os
from scipy.stats import dirichlet
import numpy as np
from scipy.stats import wishart
from scipy.stats import norm
# import numba
from scipy.stats import multivariate_normal as mvn
from numpy.linalg import inv
from numpy import log as ln
from scipy.special import psi
from numpy.random import random as rand
from scipy.stats import rv_discrete
from scipy.stats import gamma
import numpy.ma as ma


%matplotlib 


wpdf = wishart.pdf
dirpdf = dirichlet.pdf
discrete = rv_discrete

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days
Vendor:  Continuum Analytics, Inc.


Using matplotlib backend: MacOSX


Package: mkl
Message: trial mode expires in 28 days


Mixture model used to generate data:

* Model: $c_i ≈ Discrete(\pi)$, $x_i ≈ p(x|\theta_{c_i})$

* Priors: $\pi ≈ Dirichlet( \alpha, \alpha, ..., \alpha)$, $\theta_j ≈ p(\theta)$

Gaussian Mixture Model:
$\theta_j = \{ \mu_j, \Lambda_j\}$

$p(x|\theta_j) = Normal(\mu_j, \Lambda_j^{-1})$ and $p(\theta) = Normal-Wishart

**The Dirichlet process "uncovers" the correct number of clusters. Marginalization of $\pi$ will save the day in Gibbs sampling setup.**

Overall Gibbs Sampling Procedure
-----
1. For each iteration:

    a. Sample each $c_i$ given the most recent $\pi, \theta, and\ other\ c_{-i}$
    
    b. Sample each $\theta_i$ given the most recent c, $\pi$, and $\theta_{-i}$ 
    
    c. Sample $\pi$ given the most recent c and $\theta$
    
**note**: just like the earlier model in problem 2, j is iteration count over k.


**Sampling $c_i$:**
* $p(c_i = j | X, \theta, c_{-i}) p(c_i=j|c)$ 
$p(c_i = j | X, \theta, c_{-i})$ is multivariate-normal on $\mu_j$ and $\Lambda_j$

In [2]:
def sample_c_i(mu_j, lambda_j, xi, alpha, n, c_i):
    """Indicates how likely each cluster is for the data point in x_i and pre-weight.
    Returns
    ------
    A normalized K dimensional vector. This is to be used as parameter of Discrete distro 
    to sample a new index value for c_i.    
    """
    x_given_theta = mvn.pdf(xi, mu_j[j], inv(lambda_j[j])) 
    
    
def sample_theta_j(x):
    """Only depends ont he data in x_train that is assigned to cluster j. Calculate posterior in closed form
    and sample from it.
    """
    pass


def sample_pi(c):
    """ Simple Dirichlet prior-multinomial likelihood setting. Conditional posterior of pi only depends on c.
    """
    pass

_I noticed that we did not really have to calculate these variables for finite K as we assume via Gibbs sampling that these terms will be marginalized using the limits.  Instead, I simplified the problem by using the appropriate algorithm for Dirichlet Process Gaussian Mixture model. I left the above as a mental guideline for myself to keep track of where things come from._

In [3]:
def x_given_theta(xi, mu_j, lambda_j):
    return mvn.pdf(xi, mu_j[j], inv(lambda_j[j])) 


def normal_wishart(x_train, c, m, s, a, B):
    """ This refers to the p($\theta$) term in the model
    Implements the normal wishart distribution discussed in class.
    
    Returns
    -------
    
    m_j: float
        Updated mean of normal
    c_j: float
        Updated variance of normal
    a_j: float
        Updated a of the Wishart
    b_j: float
        Updated b of the wishart distribution
    
    """
    term1 = m_j = c / (s + c) * m 
    term2 = 0
    for i in range(s):
        term2 += x_train.values[i]
    m_j = term1  + 1 / (s + c) * term2     
    c_j = s + c
    a_j = a + s
    term3 = 0
    x_h = 0
    
    for _ in range(s):
        x_h += x_train.values[i]
    x_h = 1 / s * x_h
    tr = s / (a * s + 1) * (x_train.values - m).dot((x_train.values - m).T)
    for i in range(s):
        tl = x_train.values[i] - x_h
        term3 +=  tl.dot(tl.T) + tr
    b_j = B + term3
    return m_j, c_j, a_j, b_j

In [10]:
os.chdir('/Users/arkilic/Desktop/')

# Step 1

x_train = pd.read_csv('data.txt', header=None)
n, d = x_train.shape
c = 1/10
a = d
A = np.cov(x_train.values)
B = c * d * A
alpha = 1
T = 500
k = 3
s = n
m = np.mean(x_train.values)
c_i = np.ones(k)
mu_j = asarray([rand(d) for i in xrange(k)])
lambda_j = asarray([np.diag(rand(d)) for i in xrange(k)])
n_j = np.ones(n)
phi_i = np.random.rand(n)


for i in range(n):
    
    # Step 1, part (a)
    mask = np.zeros(n)
    mask[i] = 1
    mx = ma.masked_array(n_j, mask=mask)
    xi = x_train.values[i]
    tmp1 = mvn.pdf(xi, mu_j[j], inv(lambda_j[j]))
    l = mx > 0
    pos_nj_index = l.nonzero()[0]
    filler_val = tmp1 * mx / (alpha + n - 1) 
    phi_i.flat[pos_nj_index] = filler_val
    
    # Step 1, part (b)
    expectation = 0

    normal_wishart(x_train = x_train, c=c, m=m, s=s, a=a, B=B)
    
        
m_j, c_j, a_j, b_j = normal_wishart(x_train=x_train, c=c, m=m, s=s, a=a, B=B)



ValueError: Array 'mean' must be a vector of length 6.